Project title - Building a Data pipeline using Scapy and visualizing senders and receivers using Neo4j

Industry Problem 
Computer Networks Lab - UE20CS253

Deepthi Dayanand - PES1UG20CS120
Ashish Kulkarni - PES1UG20CS085
Bhumika Nayak - PES1UG20CS104

In [ ]:
import sys
!{sys.executable} -m pip install pypcapfile  #Handling pcap files
!{sys.executable} -m pip install scapy  #Used alongside pypcapfile to dissect network packets
!{sys.executable} -m pip install py2neo  #Client library for Neo4j
!{sys.executable} -m pip install binascii #Convert bw Binary and Ascii 

In [ ]:
# importing the packages we need from "pcapfile"
from pcapfile import savefile
from pcapfile.protocols.linklayer import ethernet
from pcapfile.protocols.network import ip

In [ ]:
from scapy.all import rdpcap # rdpcap loads entire pcap file to memory

In [ ]:
from py2neo import Graph, Node, Relationship # import the modules needed from py2neo package

In [ ]:
pcapfile = open('pcapfile.dump', 'rb')  # read pcap file in binary format

capfile = savefile.load_savefile(pcapfile, layers=2, verbose=True)  # load the pcapfile into a variable

Analysing data loaded using Extrapolatory Data Analysis

In [ ]:
packets = rdpcap("pcapfile.dump")  # Reading dump file using scapy 
print(packets)

In [ ]:
packets.summary()   # An overall summary of data in the PCAP file

In [ ]:
# Displays address type of the packet (IPv4/IPv6)
for number, packet in enumerate(packets.sessions()):
    print(capfile.packets[number].packet.payload)

In [ ]:
packets.sessions() # Detailed summary of packets

In [ ]:
neo4jGraph = Graph(password="123") # Creating a graph to connect to the localhost Neo4j database.

In [ ]:
for number, packet in enumerate(packets.sessions()):
    pkt = packet.split() #splitting packet object to extract information
    
    # check the first element of the list to see what type it is
    if "TCP" in pkt[0]:
        firstNode = Node("Host", name=pkt[1].split(":")[0])
        secondNode = Node("Host", name=pkt[3].split(":")[0])

        # check if it's IPv4 or IPv6
        # convert the IP object into a string, splitting it and reading the first element of the array
        if(str(capfile.packets[0].packet.payload).split()[0] == 'ipv4'):
            SENDtcp = Relationship.type("TCP - IPv4") # create a relationship of type TCP - IPv4
        else:
            SENDtcp = Relationship.type("TCP - IPv6") # create a relationship of type TCP - IPv6
        
        neo4jGraph.merge(SENDtcp(firstNode, secondNode), "Host", "name") # connecting the relationship between the two nodes created above and loading into the database
        
    # check the first element of the list to see what type it is
    elif "UDP" in pkt[0]:
        # the second element in the packet object list is the sender address
        firstNode = Node("Host", name=pkt[1].split(":")[0])
        
         # the fourth element in the packet object list is the sender address
        secondNode = Node("Host", name=pkt[3].split(":")[0])
        
        # convert the UDP packet object into a string, splitting it and reading the first element of the array
        SENDudp = Relationship.type("UDP")
        
        # connect the relationship between the two nodes created above (loading into the database)
        neo4jGraph.merge(SENDudp(firstNode, secondNode), "Host", "name")
        
        
    # check the first element of the list to see what type it is
    elif "ICMP" in pkt[0]:
        # the second element in the packet object list is the sender address
        firstNode = Node("Host", name=pkt[1].split(":")[0])
        # the fourth element in the packet object list is the sender address
        
        secondNode = Node("Host", name=pkt[3].split(":")[0])
        
        # convert the ICMP packet object into a string, splitting it and reading the first element of the array
        SENDicmp = Relationship.type("ICMP")
        
         # connect the relationship between the two nodes created above (loading into the database)
        neo4jGraph.merge(SENDicmp(firstNode, secondNode), "Host", "name")
        
    # check the first element of the list to see what type it is
    elif "ARP" in pkt[0]:
        # the second element in the packet object list is the sender address
        firstNode = Node("Host", name=pkt[1].split(":")[0])
        
        # the fourth element in the packet object list is the sender address
        secondNode = Node("Host", name=pkt[3].split(":")[0])
        
        # convert the ARP packet object into a string, splitting it and reading the first element of the array
        SENDarp = Relationship.type("ARP")
        
        # connect the relationship between the two nodes created above (loading into the database)
        neo4jGraph.merge(SENDarp(firstNode, secondNode), "Host", "name")
